# ACS Population Dataset generation for the MEP 3 process

This notebook creates a dataset of geometries tagged with ACS population estimates.
It then proportions these population values into a RouteE Compass query dataset containing
h3 hexes, so that the population assignments per hex are known before running the 
MEP 3 application.

## 1. ACS/TIGRIS Dataset Merge

here we combine ACS population data for all census block groups
with the block group geometries provided by TIGRIS and write them to a new CSV file.

### preconditions

previous to this notebook i quickly merged all TIGRIS block groups for 2022
into a single CSV file:

```python
import pandas as pd
import os
bg_dir = "/Users/rfitzger/data/census/tigris/2022-block-group"
files = os.listdir(bg_dir)
dfs = [pd.read_csv(f'{bg_dir}/{f}') for f in files if f.startswith("census")]
df = pd.concat(dfs)
df.to_csv(f"{bg_dir}/census_bg_2022.csv.gz")
```

In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
import os

# load acs data dump and block group geometries
bg = pd.read_csv("/Users/rfitzger/data/census/tigris/2022-block-group/census_bg_2022.csv.gz", dtype={"GEOID": str})
bg['geometry'] = bg.geometry.apply(wkt.loads)
bg = gpd.GeoDataFrame(bg, crs="EPSG:4326")
acs = pd.read_csv("/Users/rfitzger/data/census/acs/acs-2022-B01001/ACSDT5Y2022.B01001-Data.csv.gz", dtype={"GEO_ID": str})

In [2]:
# cleanup
# remove '1500000US' prefix from ACS GEOID
acs['GEOID'] = acs['GEO_ID'].apply(lambda s: s[9:])

In [3]:
# join geometries with population data
df = (
    acs[['GEOID', 'B01001_001E']]
        .rename(columns={"B01001_001E": "population"})
        .set_index("GEOID")
        .join(bg[['geometry', 'GEOID']].set_index("GEOID"), how="left")
).reset_index(drop=False)
df = gpd.GeoDataFrame(df, crs="EPSG:4326")

In [31]:
# any empty geometries (also implicitly catching any null values) after join?
assert all(df.geometry.apply(lambda g: g.is_empty)) == False, "invalid join"

In [34]:
# sanity check:
# Jefferson County, Colorado FIPS code is 08059
df[df["GEOID"].str.startswith("08059")].explore("population")

In [35]:
df.to_csv("/Users/rfitzger/dev/nrel/mep/mep-routee-compass/mep-routee-compass/data/national_acs_2022_population_bg.csv.gz")

## 2. population proportioning

these population estimates by block group must be proportioned into the uber h3 hexes we are using for our queries.

In [4]:
import json
import h3
from shapely.geometry import Polygon
from shapely.ops import transform
from shapely import wkt
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import gzip


In [5]:
# load queries (here would be where to load different hex grids at different resolutions)
with open("/Users/rfitzger/dev/nrel/mep/mep-routee-compass/mep-routee-compass/queries/us_h3_8.json", "r") as f:
    queries = json.loads(f.read())

In [6]:
# get area in meters for each census block group
df['area'] = df.to_crs("EPSG:3857").geometry.apply(lambda g: g.area)

In [15]:
def get_hex_polygon(hex_id):
    boundary = h3.h3_to_geo_boundary(hex_id)
    polygon = transform(lambda x, y: (y, x), Polygon(boundary))
    return polygon

def proportion_population(p: Polygon):
    rows = df.iloc[df.sindex.query(p)]
    overlap = rows.overlay(gpd.GeoDataFrame({"geometry": [p]}, crs="EPSG:4326"))
    
    # protect against empty results
    if len(overlap) == 0:
        return 0.0
    
    # weighted sum of population by overlap percentage
    overlap['pct'] = overlap.to_crs("EPSG:3857").apply(lambda r: r.geometry.area / r['area'], axis=1)
    overlap['pop_w'] = overlap.population * overlap.pct
    pop = overlap.pop_w.sum()

    return pop


In [22]:
# initialize job
pop_file = Path("/Users/rfitzger/dev/nrel/mep/mep-routee-compass/mep-routee-compass/queries/population.csv.gz")
if pop_file.exists():
    with gzip.open(pop_file, 'rb') as f:
        rows_recorded = -1  # offset to ignore the header
        for _ in f:
            rows_recorded += 1
    print(f"found {rows_recorded} rows already recorded in population.csv.gz")
else:
    rows_recorded = 0

found 12500000 rows already recorded in population.csv.gz


In [24]:
batch_size = 25_000 # ~ 480 batches
# rows_recorded = 2_525_000 # current rows in output population
# batch_id = 0
# start_batch = rows_recorded / batch_size
out_data = []


if pop_file.exists():
    with gzip.open(pop_file, 'rb') as f:
        rows_recorded = -1  # offset to ignore the header
        for _ in f:
            rows_recorded += 1
    print(f"found {rows_recorded} rows already recorded in population.csv.gz")
else:
    rows_recorded = 0

for idx, query in tqdm(enumerate(queries), total=len(queries)):
    
    # this job may be interrupted; if so, the row count from the file
    # should be updated above in the rows_recorded variable 
    # current_batch_id = int(idx / batch_size)
    if idx == rows_recorded:
        print(f"beginning at row with index {idx}")
    if idx >= rows_recorded:
                
        # manage batch saves
        save_point = len(out_data) == batch_size
        if save_point:
            out_df = pd.DataFrame(out_data)
            if pop_file.exists():
                out_df.to_csv(pop_file, mode="a", index=False, header=False)
            else:
                out_df.to_csv(pop_file, index=False)
            out_data = []
        
        # append i'th row
        hex_polygon = get_hex_polygon(query['hex_id'])
        out_data.append({"hex_id": query['hex_id'], "population": proportion_population(hex_polygon), "geometry": hex_polygon})

# append final batch
out_df = pd.DataFrame(out_data)
if pop_file.exists():
    out_df.to_csv(pop_file, mode="a", index=False, header=False)
else:
    out_df.to_csv(pop_file, index=False)
out_data = []

found 12500000 rows already recorded in population.csv.gz


 99%|█████████▊| 12328343/12501226 [00:04<00:00, 2640005.85it/s]

beginning at row with index 12500000


100%|██████████| 12501226/12501226 [00:32<00:00, 389001.41it/s] 


In [41]:
# df = pd.read_csv(pop_file)
qdf = pd.read_csv(pop_file)

In [44]:
origin_pop = df.population.sum()
result_pop = qdf.population.sum()
result_pop_pct = result_pop / origin_pop
print(
    f"after proportioning census population into h3 hexes, we now have "
    f"{result_pop}/{origin_pop} ({100*result_pop_pct:.2f}%) population "
    "assigned to the h3 grid."
)

after proportioning census population into h3 hexes, we now have 332604608.6277591/334369975 (99.47%) population assigned to the h3 grid.


In [46]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
qdf.population.describe()

count   12501226.000
mean          26.606
std          199.602
min            0.000
25%            0.120
50%            0.853
75%            5.985
max        42151.937
Name: population, dtype: float64

In [30]:
# assign population values back to query dataset
indices = []
for idx, row in qdf.iterrows():
    queries[idx]['population'] = row['population']


In [85]:
# write updated query dataset with population entries
with open("/Users/rfitzger/dev/nrel/mep/mep-routee-compass/mep-routee-compass/queries/us_h3_8_acs2022.json", "w") as f:
    f.write(json.dumps(queries, indent=4))

### visualizations of population data by state

In [48]:
qdf['state_code'] = list(map(lambda r: r['state_code'], queries))

In [59]:
out_dir = Path("/Users/rfitzger/dev/nrel/mep/mep-routee-compass/mep-routee-compass/queries/us_h3_8_pop_plot")
if not out_dir.exists():
    out_dir.mkdir()

In [81]:
# make a plot for each state
import matplotlib.pyplot as plt
import contextily as ctx

state_codes = qdf.state_code.unique()
for state_code in state_codes:
    try:
        fig, ax = plt.subplots()
        sdf = qdf[qdf['state_code']==state_code].copy()
        sdf['geometry'] = sdf.geometry.apply(wkt.loads)
        sdf = gpd.GeoDataFrame(sdf, crs="EPSG:4326")
        sdf.to_crs("EPSG:3857").plot("population", ax=ax, alpha=1, cmap="inferno_r", edgecolor="none", linewidth=0, legend=True)
        total = sdf.population.sum()
        # ax.set_title(f"{state_code}: {total:,.2f} persons")
        ax.set_xticks([])
        ax.set_yticks([])
        ctx.add_basemap(ax=ax, source=ctx.providers.OpenStreetMap.Mapnik, attribution="")
        fig.savefig(out_dir / f"{state_code}-p={total:.0f}.png", dpi=300, bbox_inches="tight")
        plt.close()
    except Exception as e:
        print(f'plot for {state_code} failed: {e}')